# Setup dependencies

**Before you begin, you will need your API Token from AI Hub.**  
To get this value, log into AI Hub and copy it from [here](https://app.aihub.qualcomm.com/account/).  
Then open the `aihub_api_token.txt` file and paste it there.  
Use CTRL-S to save the file and close it.

In [ ]:
%%time
import os
import sys

# Fix error:
# ImportError: libGL.so.1: cannot open shared object file: No such file or directory
!sudo apt-get install -q -y libgl1

# Fix error:
# TensorFlow SavedModel: export failure ❌ 167.2s: libusb-1.0.so.0: cannot open shared object file: No such file or directory
!sudo apt-get install -q -y libusb-1.0-0-dev

# Install and configure qai-hub for Quantize steps
!{sys.executable} -m pip install -q onnxruntime

# Clone Ultralytics YOLOv5
![ ! -d "yolov5" ] && git clone https://github.com/ultralytics/yolov5 -b master
# pin to known SHA on 2024-11-25 for reproducability
!cd yolov5 && git reset --hard 882c35fc43a4d7175ff24e8e20b0ec0636d75f49

# Install yolo and dependencies
!{sys.executable} -m pip install -q -r yolov5/requirements.txt

# Configure settings

The following block sets the configuration for building the model.

NOTES:
- To setup the **FULL** training session change `SAMPLE_ONLY = False`.  Training will take a long time once all of the classes are enabled.
- If you run tests with `SAMPLE_ONLY = True` and then change to `SAMPLE_ONLY = False`, **you will need to re-run "Download and prepare the dataset"**.
- **If you restart the kernel this block MUST always be re-run.**

In [ ]:
# Limit the dataset to 5 classes in order to test training quickly, change to False for full training
SAMPLE_ONLY = True
if SAMPLE_ONLY:
    CLASS_FILTER = [17, 36, 47, 68, 73]
else:
    CLASS_FILTER = []
CLASS_LIMIT = 80 # gst-plugin-mlvdetection currently rejects a model with more than 80 classes
EXPORT_CLASS_LIMIT = 4 # images per class to be used for export calibration

# Dataset settings
DATASET_NAME = "CUB_200_2011"
DATASET_FILENAME = DATASET_NAME + ".tgz"
LABELS_FILENAME = DATASET_NAME + ".labels"
LABELS_COLOR = "0x00FF00FF"
DATA_DIR = DATASET_NAME + "/"

# Model and training settings
MODEL_NAME = "yolov5m"
MODEL_FILENAME = f"{MODEL_NAME}.pt"
MODEL_INPUT_PIXEL_SIZE = 320
if SAMPLE_ONLY:
    TRAINING_EPOCHS = 250
else:
    TRAINING_EPOCHS = 2000

# Download and prepare the dataset

The dataset used for this project is: Caltech-UCSD Birds-200-2011 (CUB-200-2011)  
More information on this dataset can be found [here](https://www.vision.caltech.edu/datasets/cub_200_2011/).

To prepare the dataset for training use:
- Several text files are parsed for image data and combined into a DataFrame
- Using the `training` field, the images are split into different folders for training and validation
- A dataset configuration file under the `datasets/` folder is created to describe where the images are and the related class names
- A labels file containing class data is generated for use on the device

In [ ]:
%%time
![ -z "$MODEL_NAME" ] && echo "ERROR!! No model settings re-run \"Configure settings\" step above!"

import os
import pandas as pd
import shutil

from PIL import Image

def convert_coco_to_yolo(img_size, bbox):
    x_center = (2*bbox[0] + bbox[2])/(2*img_size[0])
    y_center = (2*bbox[1] + bbox[3])/(2*img_size[1])
    width = bbox[2]/img_size[0]
    height = bbox[3]/img_size[1]
    return (round(x_center, 6), round(y_center, 6), round(width, 6), round(height, 6))

def append_file(filename, line):
    with open(filename, "a") as file:
        file.write(line)
        file.close()

print("Downloading CUB_200_2011 files ...")
![ ! -f "$DATASET_FILENAME" ] && [ ! -d "$DATA_DIR" ] && wget --no-check-certificate -q -O $DATASET_FILENAME https://data.caltech.edu/records/65de6-vp158/files/CUB_200_2011.tgz?download=1

print("Extracting CUB_200_2011 files ...")
# Unzip and cleanup the old compressed file
![ ! -d "$DATA_DIR" ] && tar -xf $DATASET_FILENAME

print("Clearing old configured dataset flles ...")

# Remove data archive
!rm -rf $DATASET_FILENAME

# read main list of images
df = pd.read_csv(DATA_DIR + "images.txt", sep=' ',
                 names=["id", "filepath"])
# merge list of image_id to class labels
df = df.merge(pd.read_csv(DATA_DIR + "image_class_labels.txt", sep=' ',
                          names=["id", "class_id"]), on="id")
# merge list of image_id to training flag
df = df.merge(pd.read_csv(DATA_DIR + "train_test_split.txt", sep=' ',
                          names=["id", "training"]), on="id")
# merge list of image_id to bounding box data
df = df.merge(pd.read_csv(DATA_DIR + "bounding_boxes.txt", sep=' ',
                          names=["id", "x_min", "y_min", "width", "height"]), on="id")

classes_df = pd.read_csv(DATA_DIR + "classes.txt", sep=' ',
                         names=["class_id", "class_name"])
df = df.merge(classes_df, on="class_id")

print("Generating dataset and label files ...")

# Create dataset and calibration folders
!rm -rf datasets/
!mkdir -p datasets/$DATASET_NAME/images/test
!mkdir -p datasets/$DATASET_NAME/images/train
!mkdir -p datasets/$DATASET_NAME/images/val
!mkdir -p datasets/$DATASET_NAME/labels/train
!mkdir -p datasets/$DATASET_NAME/labels/val

# Remove old labels file
!rm -rf $LABELS_FILENAME

# copy the dataset template
!cp CUB_200_2011.yaml.template datasets/CUB_200_2011.yaml

class_ids = sorted(df['class_id'].drop_duplicates())
class_counter = -1
for c_id in class_ids:
    if len(CLASS_FILTER) == 0 or c_id in CLASS_FILTER:
        class_counter += 1
        if class_counter >= CLASS_LIMIT:
            break

        # Parse the class name
        c_name = classes_df[classes_df['class_id'] == c_id]['class_name'].array[0].split(".")[1]
        print(f"Parsing: {c_name}")
        # append to the dataset config
        append_file(f"datasets/{DATASET_NAME}.yaml", f"  {class_counter}: {c_name}\n")
        # append to the labels file
        append_file(LABELS_FILENAME, f'(structure)"{c_name.replace(" ", "-").replace("_", "-").lower()},id=(guint)0x{class_counter:0>4X},color=(guint){LABELS_COLOR};"\n')

        for image_id in df[df['class_id'] == c_id]['id']:
            image_dfs = df[df['id'] == image_id]
            filepath_orig = image_dfs['filepath'].array[0]
            filename_new = filepath_orig.split("/")[1]
            label_filename = filename_new.split(".")[0] + ".txt"

            # convert from bounding box data to YOLO style x_center,y_center,w,h box
            img_size = Image.open(f"{DATASET_NAME}/images/{filepath_orig}").size
            bbox = (image_dfs['x_min'].array[0], image_dfs['y_min'].array[0], image_dfs['width'].array[0], image_dfs['height'].array[0])
            yolo_box = convert_coco_to_yolo(img_size, bbox)

            if image_dfs['training'].array[0] == 1:
                loc = "train"
            if image_dfs['training'].array[0] == 0:
                loc = "val"

            # TODO: copy -> rename
            shutil.copy(f"{DATASET_NAME}/images/{filepath_orig}", f"datasets/{DATASET_NAME}/images/{loc}/{filename_new}")
            # Create label file: <class_id> <x_center> <y_center> <width> <height>
            with open(f"datasets/{DATASET_NAME}/labels/{loc}/{label_filename}", "w") as label_file:
                label_file.write(f"{class_counter} {yolo_box[0]} {yolo_box[1]} {yolo_box[2]} {yolo_box[3]}\n")

# Create calibration data (in JPG format -- later it will be converted to RAW)
!rm -rf calibration/
!mkdir -p calibration/

# For SAMPLE_ONLY we use the limited val folder data as calibration for quantize operations
# For !SAMPLE_ONLY the val data is too large: create export dir with only 4 calibration images per class type (For 80 classes == ~320 images)
src_images_dir = f"datasets/{DATA_DIR}/images/val/"
dst_images_dir = f"calibration/"

# Setup export directory for calibration data
print("Generate calibration data ...")
last_class = ""
translation_table = {ord(char): None for char in "_0123456789"}
image_list = os.listdir(src_images_dir)
export_counter = 0
for image_path in sorted(image_list):
    if image_path.endswith(".jpg"):
        export_counter += 1
        unique_name = image_path.translate(translation_table).lower()
        if unique_name != last_class:
            export_counter = 1
            last_class = unique_name
        if SAMPLE_ONLY or export_counter <= EXPORT_CLASS_LIMIT:
            shutil.copy(os.path.join(src_images_dir, image_path), dst_images_dir)

# Create calibration download
!tar -czf calibration.tar.gz calibration/

# Download test images
!wget --no-check-certificate -q -O datasets/$DATASET_NAME/images/test/multi-goldfinch-1.jpg https://t3.ftcdn.net/jpg/01/44/64/36/500_F_144643697_GJRUBtGc55KYSMpyg1Kucb9yJzvMQooW.jpg
!wget --no-check-certificate -q -O datasets/$DATASET_NAME/images/test/northern-flicker-1.jpg https://upload.wikimedia.org/wikipedia/commons/5/5c/Northern_Flicker_%28Red-shafted%29.jpg
!wget --no-check-certificate -q -O datasets/$DATASET_NAME/images/test/northern-cardinal-1.jpg https://cdn.pixabay.com/photo/2013/03/19/04/42/bird-94957_960_720.jpg
!wget --no-check-certificate -q -O datasets/$DATASET_NAME/images/test/blue-jay-1.jpg https://cdn12.picryl.com/photo/2016/12/31/blue-jay-bird-feather-animals-b8ee04-1024.jpg
!wget --no-check-certificate -q -O datasets/$DATASET_NAME/images/test/hummingbird-1.jpg http://res.freestockphotos.biz/pictures/17/17875-hummingbird-close-up-pv.jpg

# Remove the original data now that we've created the dataset
!rm -rf $DATA_DIR

print("Done!")

# Train and validate our model

If the `runs/` directory is removed (for cleaning), be sure to:
- Restart the instance kernel
- Re-run the "Configure settings" section

In [ ]:
%%time
![ -z "$MODEL_NAME" ] && echo "ERROR!! No model settings re-run \"Configure settings\" step above!"

import shutil
import sys

MODEL_FILENAME_RELPATH = f"../{MODEL_FILENAME}"
DATA_CONFIG_RELPATH = f"../datasets/{DATASET_NAME}.yaml"

# Train the model
!cd yolov5/ && python train.py --weights $MODEL_FILENAME_RELPATH --data $DATA_CONFIG_RELPATH --epochs $TRAINING_EPOCHS --imgsz $MODEL_INPUT_PIXEL_SIZE

# Validate the model
!cd yolov5/ && python val.py --weights $MODEL_FILENAME_RELPATH --data $DATA_CONFIG_RELPATH --imgsz $MODEL_INPUT_PIXEL_SIZE

# Backup best.pt after training
shutil.copy(f"yolov5/runs/train/exp/weights/best.pt", f"{MODEL_FILENAME}")
print(f"Copied best weights as: {MODEL_FILENAME}")

# Run inference using test images and unquantized model

If the `runs/` directory is removed (for cleaning), be sure to:
- Restart the instance kernel
- Re-run the "Configure settings" section

The results of the image tests are stored under the `yolov5/runs/detect/exp` folder.

In [ ]:
%%time
![ -z "$MODEL_NAME" ] && echo "ERROR!! No model settings re-run \"Configure settings\" step above!"

DATA_TEST_RELPATH = f"../datasets/{DATASET_NAME}/images/test/"
DATA_CONFIG_RELPATH = f"../datasets/{DATASET_NAME}.yaml"
MODEL_FILENAME_RELPATH = f"../{MODEL_FILENAME}"

!cd yolov5/ && python detect.py --weights $MODEL_FILENAME_RELPATH --imgsz $MODEL_INPUT_PIXEL_SIZE --data $DATA_CONFIG_RELPATH --source $DATA_TEST_RELPATH

# Export to TFLite format

In [ ]:
%%time
![ -z "$MODEL_NAME" ] && echo "ERROR!! No model settings re-run \"Configure settings\" step above!"

DATA_CONFIG_RELPATH = f"../datasets/{DATASET_NAME}-export.yaml"
MODEL_FILENAME_RELPATH = f"../{MODEL_FILENAME}"

!cd yolov5/ && python export.py --include tflite --weights $MODEL_FILENAME_RELPATH --imgsz $MODEL_INPUT_PIXEL_SIZE --data $DATA_CONFIG_RELPATH

print("Done!")
print(f"Download model file: {MODEL_NAME}_fp16.tflite")
print(f"Download labels file: {LABELS_FILENAME}")
print("Download calibration images: calibration.tar.gz")